## Converting netCDF Data into PCraster Timeseries format.
Coded by: Dinesh Joshi \
Email: joshidinesh0227@gmail.com

## Provide Inputs in below.

In [1]:
# varibale name for your data 
myvar_Prec="P"
myvar_Tmax="Tmax"
myvar_Tmin="Tmin"
#Spatial Extent
Xmin=182010
Xmax=275981
Ymin=3089163
Ymax=3204333

###  In the next code cell, we will load the netcdf data and convert it into pcraster timeseries format in the desired resolution as mask file. Run the cell below to execute this code. You can check Prec, Tmax, Tmin subfolders inside the main folder for the resultant data. The processing will take time depending on the amount of data and size of study area.

In [5]:
# importing necessary libraries
import netCDF4
from netCDF4 import Dataset
import numpy as np
from pcraster import *
from pcraster.framework import *

# Importing NetCDF file
import glob
Prec_files = glob.glob(r"./Netcdf Prec/*.nc")
Tmax_files = glob.glob(r"./Netcdf Tmax/*.nc")
Tmin_files = glob.glob(r"./Netcdf Tmin/*.nc")

netcdf= Dataset(Prec_files[0]) # opening netcdf file at index 0

# Extracting the indices of the coordinates within the Study Area¶
latlog_index=[]
for latitude in range(len(netcdf.variables["latitude"][:])):
    for longitude in range(len(netcdf.variables["longitude"][:])):
        a=float(netcdf.variables["latitude"][latitude])
        b=float(netcdf.variables["longitude"][longitude])
        if Xmin <= b <= Xmax and  Ymin <= a <= Ymax:
            latlog_index.append((longitude,latitude))
Data=[]
for i in range(len(latlog_index)):
    data=netcdf.variables[myvar_Prec][0,latlog_index[i][1],latlog_index[i][0]]
    latitude=netcdf.variables["latitude"][latlog_index[i][1]]
    longitude=netcdf.variables["longitude"][latlog_index[i][0]]
    Data.append((data,longitude,latitude))

# Creating a station.txt file with coordinates id
with open('coordinates.txt', 'w') as C:
        for i in range(len(Data)):
            C.write(str(Data[i][1]) + ' ' + str(Data[i][2]) + ' ' + str(i+1))
            C.write("\n")

# Creating  time series scaler data text files
def timeseriesTSS(myvar,files):
    with open(myvar+'TimeseriesData.tss', 'w') as f:
        f.write(myvar + " time series scaler data")
        f.write("\n")
        f.write(str(len(latlog_index)+1))
        f.write("\n")
        f.write("timestep")
        f.write("\n")
        for i in range(len(latlog_index)):
                f.write(str(i))
                f.write("\n")
        Timesteps=1
        for file in range(len(files)):
            for time in range(1,len(Dataset(files[file]).variables["time"][:])+1):
                Data=[]
                for i in range(len(latlog_index)):
                    data=Dataset(files[file]).variables[myvar][time-1,latlog_index[i][1],latlog_index[i][0]]
                    Data.append(data)
                f.write(str(Timesteps)+' ')
                for i in range(len(latlog_index)):
                    f.write(str(Data[i])+ ' ')
                f.write(str("\n"))
                Timesteps=Timesteps+1
timeseriesTSS(myvar_Prec,Prec_files)
timeseriesTSS(myvar_Tmax,Tmax_files)
timeseriesTSS(myvar_Tmin,Tmin_files)

# number of mapfiles
LastTimestep = sum([len(Dataset(file).variables["time"][:]) for file in Prec_files])

# change the current working directory to GDM Training'
import os
os.chdir("..")
os.chdir("..")
os.getcwd()

# Creating a pcraster map file for point coordinates¶
# set the paths to the input and output files
coordinates_file = './Data Preprocessing/Netcdf Data/coordinates.txt'
mask_file = './Inputs/mask.map'
output_file = './Data Preprocessing/Netcdf Data/coordinates.map'

# build the command string with the file paths
command = ['col2map', '-N', coordinates_file, output_file, '--clone', mask_file]

# execute the command using subprocess.run
subprocess.run(command)

# Interpolation using Inverse distance weighting
class InterpolateRainfall(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Map with coordinates
        self.coordinates = self.readmap("./Data Preprocessing/Netcdf Data/coordinates")
        # Boolean mask for IDW 
        self.mask = self.readmap("./Inputs/mask")  
        #variable for tss file
        self.Prec_DataTSS = "./Data Preprocessing/Netcdf Data/"+myvar_Prec+'TimeseriesData.tss' 
        self.Tmax_DataTSS = "./Data Preprocessing/Netcdf Data/"+myvar_Tmax+'TimeseriesData.tss'
        self.Tmin_DataTSS = "./Data Preprocessing/Netcdf Data/"+myvar_Tmin+'TimeseriesData.tss'
       
    def dynamic(self):
        # Reading precipitation at coordinates and make dynamic map stations
        Prec_DataAtStation = timeinputscalar(self.Prec_DataTSS,self.coordinates)
        #IDW interpolation
        Prec_DataIDW = inversedistance(self.mask,Prec_DataAtStation,2,0,0)
        
        # Reading precipitation at coordinates and make dynamic map stations
        Tmax_DataAtStation = timeinputscalar(self.Tmax_DataTSS,self.coordinates)
        #IDW interpolation
        Tmax_DataIDW = inversedistance(self.mask,Tmax_DataAtStation,2,0,0)
        
        # Reading precipitation at coordinates and make dynamic map stations
        Tmin_DataAtStation = timeinputscalar(self.Tmin_DataTSS,self.coordinates)
        #IDW interpolation
        Tmin_DataIDW = inversedistance(self.mask,Tmin_DataAtStation,2,0,0)
        
        self.report(Prec_DataIDW,"./Prec/prec")
        self.report(Tmax_DataIDW,"./Tmax/tmax")
        self.report(Tmin_DataIDW,"./Tmin/tmin")
        
myModel = InterpolateRainfall("./Inputs/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=LastTimestep, firstTimestep=1) #Adjust first and last time step
dynModelFw.run()
os.chdir("./Data Preprocessing/Netcdf Data")

..................................................................

KeyboardInterrupt: 